In [6]:
!pip install contractions

In [32]:
import pandas as pd
import os
import nltk
# nltk.download('punkt')
nltk.download('stopwords')
# os.system('pip install contractions')
import re
import contractions
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
import collections
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\awskr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url = "https://raw.githubusercontent.com/mturner49/8600-deep-learning-project/main/data/CNN_Articels_clean.csv"

data = pd.read_csv(url)
data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Index            4076 non-null   int64 
 1   Author           4076 non-null   object
 2   Date published   4076 non-null   object
 3   Category         4076 non-null   object
 4   Section          4076 non-null   object
 5   Url              4076 non-null   object
 6   Headline         4076 non-null   object
 7   Description      4076 non-null   object
 8   Keywords         4076 non-null   object
 9   Second headline  4076 non-null   object
 10  Article text     4076 non-null   object
dtypes: int64(1), object(10)
memory usage: 350.4+ KB


In [3]:
df = data.copy()


In [4]:
keep_cols = ['Article text']

for cols in df.columns:
    if cols not in keep_cols:
        df = df.drop(columns=cols)


In [7]:
df['Article text'] = df['Article text'].apply(lambda x: re.sub(r'\[[0-9]*\]', ' ', str(x)))
df['Article text'] = df['Article text'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

In [8]:
df['no_contractions'] = df['Article text'].apply(lambda x: [contractions.fix(word) for word in x.split()])

In [9]:
df['formatted_text'] = df['Article text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))
df['formatted_text'] = df['formatted_text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))

In [12]:
df['tokenize_sentences'] = df['Article text'].apply(sent_tokenize)

In [16]:
stop_words = set(stopwords.words('english'))
df['formatted_no_stopwords'] = df['formatted_text'].apply(lambda x: [word for word in nltk.word_tokenize(x) if word not in stop_words])

In [17]:
df.head()

,Article text,no_contractions,formatted_text,tokenize_sentences,formatted_no_stopwords
0,"(CNN)Right now, there's a shortage of truck d...","[(CNN)Right, now,, there is, a, shortage, of, ...",CNN Right now there s a shortage of truck d...,"[ (CNN)Right now, there's a shortage of truck ...","[CNN, Right, shortage, truck, drivers, US, wor..."
1,(CNN)Working in a factory or warehouse can me...,"[(CNN)Working, in, a, factory, or, warehouse, ...",CNN Working in a factory or warehouse can me...,[ (CNN)Working in a factory or warehouse can m...,"[CNN, Working, factory, warehouse, mean, task,..."
2,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[(CNN)In, a, Hong, Kong, warehouse,, a, swarm,...",CNN In a Hong Kong warehouse a swarm of aut...,"[ (CNN)In a Hong Kong warehouse, a swarm of au...","[CNN, In, Hong, Kong, warehouse, swarm, autono..."
3,"New York (CNN Business)For many years, the wor...","[New, York, (CNN, Business)For, many, years,, ...",New York CNN Business For many years the wor...,"[New York (CNN Business)For many years, the wo...","[New, York, CNN, Business, For, many, years, w..."
4,The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...",The European Union formally approved on Tuesda...,[The European Union formally approved on Tuesd...,"[The, European, Union, formally, approved, Tue..."


In [36]:
counter = Counter()
df['formatted_no_stopwords'].map(counter.update)
most_frequent = pd.DataFrame(counter.most_common())
most_frequent.columns = ['token', 'count']
most_frequent.head(20)

,token,count
0,I,23985
1,The,23833
2,said,19513
3,CNN,9743
4,people,8086
5,It,7875
6,one,7548
7,year,7164
8,first,7096
9,time,6771
